In [1]:
from dabry.problem import NavigationProblem
from dabry.wind import DiscreteWind
from dabry.model import ZermeloGeneralModel
from dabry.misc import Utils
from dabry.ddf_manager import DDFmanager
from dabry.solver_ef import SolverEF
import numpy as np

In [2]:
i_member = 0
for i_member in range(51):
    wind = DiscreteWind()
    wind.load(f'../output/wind_scenarii_{i_member}/wind.h5')
    model = ZermeloGeneralModel(23, coords='gcs')
    model.update_wind(wind)

    x_init = Utils.DEG_TO_RAD * np.array((-55, 40))
    x_target = Utils.DEG_TO_RAD * np.array((-10, 40))

    bl = Utils.DEG_TO_RAD * np.array((-60, 33))
    tr = Utils.DEG_TO_RAD * np.array((-5, 55))

    pb = NavigationProblem(model, x_init, x_target, 'gcs', bl=bl, tr=tr)

    ddf = DDFmanager(cache_wind=True)
    ddf.setup()
    ddf.set_case(f'wind_scenarii_{i_member}')
    ddf.clean_output_dir()
    
    solver = SolverEF(pb, max_time=20*3600, max_steps=500)
    res = solver.solve()
    
    ddf.dump_trajs([res.traj])
    ddf.log(pb)

Problem | TIMEOPT | 23.00 m/s | 3.80e+06 m | scale 45h50m | orthodromic 51h58m
Steps :    169/500, Extremals :    220, Active :  132, Dist :   9 Comp : 1 Stopped empty active list
Target reached in 40h54m | -10% no wind | -21% orthodromic | cpu time 4.83s
Problem | TIMEOPT | 23.00 m/s | 3.80e+06 m | scale 45h50m | orthodromic 52h42m
Steps :    171/500, Extremals :    217, Active :  123, Dist :   8 Comp : 1 Stopped empty active list
Target reached in 41h47m | -8% no wind | -20% orthodromic | cpu time 4.94s
Problem | TIMEOPT | 23.00 m/s | 3.80e+06 m | scale 45h50m | orthodromic 51h49m
Steps :    170/500, Extremals :    218, Active :  151, Dist :   8 Comp : 1 Stopped empty active list
Target reached in 41h47m | -8% no wind | -19% orthodromic | cpu time 4.91s
Problem | TIMEOPT | 23.00 m/s | 3.80e+06 m | scale 45h50m | orthodromic 51h11m
Steps :    170/500, Extremals :    215, Active :  141, Dist :   8 Comp : 1 Stopped empty active list
Target reached in 41h11m | -10% no wind | -19% orthodr

Steps :    172/500, Extremals :    212, Active :  148, Dist :   9 Comp : 1 Stopped empty active list
Target reached in 41h47m | -8% no wind | -18% orthodromic | cpu time 5.57s
Problem | TIMEOPT | 23.00 m/s | 3.80e+06 m | scale 45h50m | orthodromic 53h47m
Steps :    169/500, Extremals :    217, Active :  127, Dist :   8 Comp : 1 Stopped empty active list
Target reached in 40h54m | -10% no wind | -23% orthodromic | cpu time 5.44s
Problem | TIMEOPT | 23.00 m/s | 3.80e+06 m | scale 45h50m | orthodromic 50h41m
Steps :    171/500, Extremals :    216, Active :  119, Dist :   8 Comp : 1 Stopped empty active list
Target reached in 40h1m | -12% no wind | -21% orthodromic | cpu time 5.11s
Problem | TIMEOPT | 23.00 m/s | 3.80e+06 m | scale 45h50m | orthodromic 51h32m
Steps :    169/500, Extremals :    220, Active :  133, Dist :   8 Comp : 1 Stopped empty active list
Target reached in 41h29m | -9% no wind | -19% orthodromic | cpu time 5.11s
Problem | TIMEOPT | 23.00 m/s | 3.80e+06 m | scale 45h50m 

In [4]:
import shutil
for i_member in range(51):
    try:
        shutil.copy(f'../output/wind_scenarii_{i_member}/trajectories.h5',
                   f'../output/wind_scenarii_all/trajectories_{i_member}.h5')
    except FileNotFoundError:
        pass
shutil.copy('../output/wind_scenarii_0/params.json', '../output/wind_scenarii_all/params.json')
shutil.copy('../output/wind_scenarii_0/wind.h5', '../output/wind_scenarii_all/wind.h5')

'../output/wind_scenarii_all/wind.h5'

In [15]:
wind = DiscreteWind()
wind.load(f'../output/wind_scenarii_all/wind.h5')
model = ZermeloGeneralModel(23, coords='gcs')
model.update_wind(wind)

x_init = Utils.DEG_TO_RAD * np.array((-55, 40))
x_target = Utils.DEG_TO_RAD * np.array((-10, 40))

bl = Utils.DEG_TO_RAD * np.array((-60, 33))
tr = Utils.DEG_TO_RAD * np.array((-5, 55))

pb = NavigationProblem(model, x_init, x_target, 'gcs', bl=bl, tr=tr)

ddf = DDFmanager(cache_wind=True)
ddf.setup()
ddf.set_case(f'wind_scenarii_all')

from dabry.feedback import HTargetFB
from dabry.stoppingcond import DistanceSC
pb.load_feedback(HTargetFB(x_target, 'gcs'))
sc = DistanceSC(lambda x: (x - x_target) @ (x - x_target), (x_init - x_target) @ (x_init - x_target) / 500)
pb.integrate_trajectory(x_init, sc, t_init=pb.model.wind.t_start, int_step=3600/100)
ddf.dump_trajs(pb.trajs, filename='trajectories_heuristic.h5')
ddf.log(pb)